In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
%%writefile task1.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

const int N = 1'000'000;
const int BLOCK_SIZE = 256;
const float MULTIPLIER = 2.5f;

/*Ядро CUDA с использованием только глобальной памяти.
 Каждый поток обрабатывает один элемент массива*/

__global__ void multiplyGlobal(float* data, float value, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] *= value;
    }
}

/*Ядро CUDA с использованием разделяемой памяти.
Данные блока загружаются в shared memory, обрабатываются и затем записываются обратно в глобальную память.*/

__global__ void multiplyShared(float* data, float value, int n) {
    __shared__ float sharedData[BLOCK_SIZE];

    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    int localIdx = threadIdx.x;

    if (globalIdx < n) {
        sharedData[localIdx] = data[globalIdx];
        sharedData[localIdx] *= value;
        data[globalIdx] = sharedData[localIdx];
    }
}

int main() {
    float* h_data = new float[N];

    for (int i = 0; i < N; i++) {
        h_data[i] = static_cast<float>(i);
    }

    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    /* Версия с глобальной памятью  */

    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    auto startGlobal = high_resolution_clock::now();
    multiplyGlobal<<<gridSize, BLOCK_SIZE>>>(d_data, MULTIPLIER, N);
    cudaDeviceSynchronize();
    auto endGlobal = high_resolution_clock::now();

    double timeGlobal =
        duration<double, milli>(endGlobal - startGlobal).count();

    /* Версия с разделяемой памятью  */

    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    auto startShared = high_resolution_clock::now();
    multiplyShared<<<gridSize, BLOCK_SIZE>>>(d_data, MULTIPLIER, N);
    cudaDeviceSynchronize();
    auto endShared = high_resolution_clock::now();

    double timeShared =
        duration<double, milli>(endShared - startShared).count();

    cout << "Размер массива: " << N << " элементов\n";
    cout << "Глобальная память: " << timeGlobal << " мс\n";
    cout << "Разделяемая память: " << timeShared << " мс\n";

    cudaFree(d_data);
    delete[] h_data;

    return 0;
}


Overwriting task1.cu


In [7]:
!nvcc task1.cu -o task1
!./task1


Размер массива: 1000000 элементов
Глобальная память: 9.07819 мс
Разделяемая память: 0.071487 мс


In [8]:
%%writefile task2.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

const int N = 1'000'000;


/*CUDA-ядро для поэлементного сложения двух массивов. Каждый поток обрабатывает один элемент.*/

__global__ void vectorAdd(const float* a, const float* b, float* c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    float* h_a = new float[N];
    float* h_b = new float[N];
    float* h_c = new float[N];

    for (int i = 0; i < N; i++) {
        h_a[i] = static_cast<float>(i);
        h_b[i] = static_cast<float>(2 * i);
    }

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    cudaMalloc(&d_c, N * sizeof(float));

    cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice);

    int blockSizes[] = {128, 256, 512};

    cout << "Размер массива: " << N << " элементов\n";

    for (int i = 0; i < 3; i++) {
        int blockSize = blockSizes[i];
        int gridSize = (N + blockSize - 1) / blockSize;

        auto start = high_resolution_clock::now();
        vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
        auto end = high_resolution_clock::now();

        double time =
            duration<double, milli>(end - start).count();

        cout << "Размер блока: " << blockSize
             << " | Время выполнения: " << time << " мс\n";
    }

    cudaMemcpy(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    return 0;
}


Writing task2.cu


In [9]:
!nvcc task2.cu -o task2
!./task2

Размер массива: 1000000 элементов
Размер блока: 128 | Время выполнения: 7.43879 мс
Размер блока: 256 | Время выполнения: 0.002794 мс
Размер блока: 512 | Время выполнения: 0.003128 мс


In [1]:
%%writefile task3.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

const int N = 1'000'000;
const int BLOCK_SIZE = 256;

/*Коалесцированный доступ: соседние потоки обращаются к соседним элементам памяти.*/

__global__ void coalescedAccess(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

/* Некоалесцированный доступ:потоки обращаются к памяти с большим шагом,что приводит к неэффективному использованию памяти.*/

__global__ void nonCoalescedAccess(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    int accessIdx = (idx * stride) % n;
    if (accessIdx < n) {
        data[accessIdx] = data[accessIdx] * 2.0f;
    }
}

int main() {
    float* h_data = new float[N];

    for (int i = 0; i < N; i++) {
        h_data[i] = static_cast<float>(i);
    }

    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    /*  Коалесцированный доступ  */

    auto startCoalesced = high_resolution_clock::now();
    coalescedAccess<<<gridSize, BLOCK_SIZE>>>(d_data, N);
    cudaDeviceSynchronize();
    auto endCoalesced = high_resolution_clock::now();

    double timeCoalesced =
        duration<double, milli>(endCoalesced - startCoalesced).count();

    /*  Некоалесцированный доступ  */

    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    auto startNon = high_resolution_clock::now();
    nonCoalescedAccess<<<gridSize, BLOCK_SIZE>>>(d_data, N);
    cudaDeviceSynchronize();
    auto endNon = high_resolution_clock::now();

    double timeNon =
        duration<double, milli>(endNon - startNon).count();

    cout << "Размер массива: " << N << " элементов\n";
    cout << "Коалесцированный доступ: " << timeCoalesced << " мс\n";
    cout << "Некоалесцированный доступ: " << timeNon << " мс\n";

    cudaFree(d_data);
    delete[] h_data;

    return 0;
}


Writing task3.cu


In [2]:
!nvcc task3.cu -o task3
!./task3

Размер массива: 1000000 элементов
Коалесцированный доступ: 57.9645 мс
Некоалесцированный доступ: 0.071138 мс


In [6]:
%%writefile task4.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

const int N = 1'000'000;

/*CUDA-ядро для поэлементного сложения массивов.Каждый поток обрабатывает один элемент. */
__global__ void vectorAdd(const float* a, const float* b, float* c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

void runTest(int blockSize, const float* d_a, const float* d_b, float* d_c) {
    int gridSize = (N + blockSize - 1) / blockSize;

    auto start = high_resolution_clock::now();
    vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, N);
    cudaDeviceSynchronize();
    auto end = high_resolution_clock::now();

    double time =
        duration<double, milli>(end - start).count();

    cout << "Размер блока: " << blockSize
         << " | Время выполнения: " << time << " мс\n";
}

int main() {
    float* h_a = new float[N];
    float* h_b = new float[N];

    for (int i = 0; i < N; i++) {
        h_a[i] = static_cast<float>(i);
        h_b[i] = static_cast<float>(2 * i);
    }

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, N * sizeof(float));
    cudaMalloc(&d_b, N * sizeof(float));
    cudaMalloc(&d_c, N * sizeof(float));

    cudaMemcpy(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice);

    cout << "Размер массива: " << N << " элементов\n";

    /* Неоптимальная конфигурация */
    cout << "\nНеоптимальная конфигурация:\n";
    runTest(64, d_a, d_b, d_c);

    /* Оптимизированные конфигурации */
    cout << "\nОптимизированные конфигурации:\n";
    runTest(256, d_a, d_b, d_c);
    runTest(512, d_a, d_b, d_c);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] h_a;
    delete[] h_b;

    return 0;
}


Overwriting task4.cu


In [7]:
!nvcc task4.cu -o task4
!./task4

Размер массива: 1000000 элементов

Неоптимальная конфигурация:
Размер блока: 64 | Время выполнения: 7.69667 мс

Оптимизированные конфигурации:
Размер блока: 256 | Время выполнения: 0.003483 мс
Размер блока: 512 | Время выполнения: 0.001782 мс


Контрольные вопросы к Assignment 3

(CUDA и архитектура GPU)

1. Какие основные типы памяти существуют в архитектуре CUDA и чем они отличаются по скорости доступа?

В архитектуре CUDA используются глобальная, разделяемая, локальная, константная и регистровая память. Регистры и разделяемая память имеют наименьшую задержку доступа и используются внутри потоков и блоков. Глобальная память обладает наибольшей задержкой, но доступна всем потокам и имеет большой объём. Константная память оптимизирована для чтения и используется при неизменяемых данных.

2. В каких случаях использование разделяемой памяти позволяет ускорить выполнение CUDA-программы?

Разделяемая память позволяет ускорить выполнение программы в случаях, когда данные многократно используются потоками одного блока. Перемещение данных из глобальной памяти в разделяемую снижает количество медленных обращений к глобальной памяти и уменьшает задержки доступа.

3. Как шаблон доступа к глобальной памяти влияет на производительность GPU-программы?

Шаблон доступа к глобальной памяти напрямую влияет на эффективность использования пропускной способности памяти. Коалесцированный доступ, при котором соседние потоки обращаются к соседним адресам памяти, обеспечивает высокую производительность. Некоалесцированный доступ приводит к увеличению числа транзакций памяти и снижению производительности.

4. Почему одинаковый алгоритм на GPU может показывать разное время выполнения при разных способах обращения к памяти?

Разное время выполнения связано с различной эффективностью использования памяти. Даже при одинаковом алгоритме неэффективный доступ к глобальной памяти может значительно увеличить задержки и снизить пропускную способность, что приводит к ухудшению общей производительности.

5. Как размер блока потоков влияет на производительность CUDA-ядра?

Размер блока потоков определяет количество потоков, выполняемых одновременно на мультипроцессоре GPU. Слишком маленький блок приводит к недостаточной загрузке вычислительных ресурсов, а слишком большой — к ограничению по доступным регистрам и разделяемой памяти. Оптимальный размер блока позволяет максимально эффективно использовать ресурсы GPU.

6. Что такое варп и почему важно учитывать его при разработке CUDA-программ?

Варп — это группа из 32 потоков, которые выполняются синхронно на GPU. Важно учитывать варпы, так как расхождение ветвлений внутри варпа (warp divergence) приводит к последовательному выполнению веток и снижению производительности.

7. Какие факторы необходимо учитывать при выборе конфигурации сетки и блоков потоков?

При выборе конфигурации необходимо учитывать архитектуру GPU, размер задачи, использование регистров и разделяемой памяти, а также требования к коалесцированному доступу к памяти. Целью является достижение максимальной загрузки вычислительных ресурсов при минимальных накладных расходах.

8. Почему оптимизация CUDA-программы часто начинается с анализа работы с памятью, а не с изменения алгоритма?

Во многих CUDA-программах производительность ограничена пропускной способностью памяти, а не вычислительными возможностями GPU. Оптимизация доступа к памяти, уменьшение количества обращений к глобальной памяти и использование разделяемой памяти часто дают больший прирост производительности, чем изменение самого алгоритма.